In [8]:
import numpy as np
import pandas as pd

In [9]:
def lagrange_basis(x, nodes):
    """
    Compute the 1D Lagrange basis polynomials for each node.
    """
    basis = []
    for i, xi in enumerate(nodes):
        li = 1.0
        for j, xj in enumerate(nodes):
            if j != i:
                li *= (x - xj) / (xi - xj)
        basis.append(li)

    return np.array(basis)

In [10]:
def tensor_product(fvals, xi, eta, xi_nodes, eta_nodes):

    l_xi = lagrange_basis(xi, xi_nodes)
    l_eta = lagrange_basis(eta, eta_nodes)

    nxi = len(xi_nodes)
    neta = len(eta_nodes)
    assert np.shape(fvals) == (nxi, neta), "wrong number of f_vals"

    # fvals_ij, i is xi indexi, j is eta indexi
    return float(l_xi @ fvals @ l_eta)

In [11]:
def bilinear_interp(fvals, xi, eta):
    xi_nodes  = [-1,  1]
    eta_nodes = [-1,  1]
    return tensor_product(fvals, xi, eta, xi_nodes, eta_nodes)

def biquadratic_interp(fvals, xi, eta):
    xi_nodes  = [-1, 0, 1]
    eta_nodes = [-1, 0, 1]
    return tensor_product(fvals, xi, eta, xi_nodes, eta_nodes)

def bicubic_interp(fvals, xi, eta):
    xi_nodes  = [-1, -1/3, 1/3, 1]
    eta_nodes = [-1, -1/3, 1/3, 1]
    return tensor_product(fvals, xi, eta, xi_nodes, eta_nodes)

In [12]:
def f(xi, eta):
    return xi**2 * eta + np.cos(np.pi * xi * eta / 2)

In [18]:
schemes = {
    'bilinear':   (bilinear_interp,   [-1,  1],        [-1,  1]),
    'biquadratic':(biquadratic_interp,[-1,  0,  1],    [-1,  0,  1]),
    'bicubic':    (bicubic_interp,    [-1, -1/3, 1/3, 1],[-1, -1/3, 1/3, 1]),
}

eva_points = [(0.5,  0.0), (0.2, -0.3)]

results = []
for name, (interpolant, xi_nodes, eta_nodes) in schemes.items():

    f_vals = []
    for xi_node in xi_nodes:
        for eta_node in eta_nodes:
            f_vals.append(f(xi_node, eta_node))

    f_vals = np.array(f_vals).reshape( len(xi_nodes), len(eta_nodes) )

    for xi_eva, eta_eva in eva_points:
        interp_val = interpolant(f_vals, xi_eva, eta_eva)
        exact_val  = f(xi_eva, eta_eva)
        err        = abs(interp_val - exact_val)
        results.append({
            'method':      name,
            'xi':          xi_eva,
            'eta':         eta_eva,
            'interp_val':  interp_val,
            'exact_val':   exact_val,
            'abs_error':   err
        })

df = pd.DataFrame(results)
df = df[['method','xi','eta','interp_val','exact_val','abs_error']]
print(df.to_string(index=False))

     method  xi  eta    interp_val  exact_val  abs_error
   bilinear 0.5  0.0  5.551115e-17   1.000000   1.000000
   bilinear 0.2 -0.3 -3.000000e-01   0.983562   1.283562
biquadratic 0.5  0.0  1.000000e+00   1.000000   0.000000
biquadratic 0.2 -0.3  9.844000e-01   0.983562   0.000838
    bicubic 0.5  0.0  9.956904e-01   1.000000   0.004310
    bicubic 0.2 -0.3  9.837117e-01   0.983562   0.000150


## Discussion

The accuracy of tensor product grid scheme highly depends on the distribution of nodes. The interpolation value of bilinear method can only reproduce the $\eta$ value, since by construction and four weights are (-1, -1, 1, 1) for the basis function, the dependence on $\xi$ is eliminated. For biquadratic method, it gives exact result for point (0.5, 0.0) just because happen to have a grid point at $\eta=0$. That forces $f(\xi,0)=\cos(0)=1$ exactly for all $\xi$, so zero error at (0.5,0).